In [ ]:
# Google Colab 환경에서 필요한 라이브러리 설치
!pip install selenium beautifulsoup4 requests webdriver-manager

# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import requests
import re

In [9]:
# Chrome 웹드라이버 설정 (Google Colab용)
def setup_chrome_driver():
    """Chrome 웹드라이버 설정"""
    # Chrome과 ChromeDriver 설치
    !apt-get update
    !apt install chromium-chromedriver

    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 헤드리스 모드
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    # User-Agent 설정
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    driver = webdriver.Chrome(options=chrome_options)

    # 웹드라이버 감지 방지
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

def extract_and_click_lg_dryer_links():
    """
    LG 전자 건조기 페이지에서 제품 링크를 추출하고 클릭하는 함수
    """
    driver = setup_chrome_driver()
    target_url = "https://www.lge.co.kr/category/dryers"

    try:
        print("=== LG 전자 건조기 페이지 접속 중 ===")
        driver.get(target_url)

        # 페이지 로딩 대기
        time.sleep(3)

        # 쿠키 동의 버튼이 있다면 클릭
        try:
            cookie_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '동의') or contains(text(), '확인') or contains(text(), 'OK')]"))
            )
            cookie_button.click()
            print("쿠키 동의 완료")
            time.sleep(2)
        except TimeoutException:
            print("쿠키 동의 버튼 없음")

        # 페이지 스크롤하여 모든 제품 로드
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # 제품 리스트 컨테이너 찾기
        try:
            # 여러 가능한 클래스명 시도
            possible_selectors = [
                "CommonPcListUnitProduct_list_unit_product__AeuOT",
                "[class*='list_unit_product']",
                "[class*='product_list']",
                ".product-item",
                ".list-item"
            ]

            products = []
            for selector in possible_selectors:
                try:
                    if selector.startswith('['):
                        products = driver.find_elements(By.CSS_SELECTOR, selector)
                    else:
                        products = driver.find_elements(By.CLASS_NAME, selector)

                    if products:
                        print(f"제품 요소 {len(products)}개 발견 (selector: {selector})")
                        break
                except:
                    continue

            if not products:
                # 페이지 소스 분석
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')

                # 제품 관련 클래스 찾기
                product_elements = soup.find_all('div', class_=re.compile(r'.*product.*|.*item.*|.*list.*', re.I))
                print(f"BeautifulSoup으로 발견한 제품 관련 요소: {len(product_elements)}개")

                # 링크가 있는 요소 찾기
                link_elements = soup.find_all('a', href=True)
                product_links = [link for link in link_elements if '/products/' in link.get('href', '') or '/dryer' in link.get('href', '')]

                print(f"제품 링크로 추정되는 요소: {len(product_links)}개")

                # 직접 링크 클릭 시도
                for i, link in enumerate(product_links[:5]):  # 최대 5개만
                    href = link.get('href')
                    if href:
                        if href.startswith('/'):
                            full_url = f"https://www.lge.co.kr{href}"
                        else:
                            full_url = href

                        print(f"\n[{i+1}] 제품 링크 방문: {full_url}")

                        # 새 탭에서 열기
                        driver.execute_script(f"window.open('{full_url}', '_blank');")
                        time.sleep(2)

                        # 새 탭으로 전환
                        driver.switch_to.window(driver.window_handles[-1])

                        # 페이지 정보 수집
                        try:
                            page_title = driver.title
                            current_url = driver.current_url
                            print(f"페이지 제목: {page_title}")
                            print(f"현재 URL: {current_url}")

                            # 제품 정보 수집 시도
                            try:
                                product_name = driver.find_element(By.CSS_SELECTOR, "h1, .product-name, [class*='title'], [class*='name']").text
                                print(f"제품명: {product_name}")
                            except:
                                print("제품명을 찾을 수 없음")

                        except Exception as e:
                            print(f"페이지 정보 수집 중 오류: {e}")

                        time.sleep(3)

                        # 탭 닫기
                        driver.close()

                        # 원래 탭으로 돌아가기
                        driver.switch_to.window(driver.window_handles[0])

                        time.sleep(2)

                return

        except Exception as e:
            print(f"제품 요소 찾기 중 오류: {e}")
            return

        # 각 제품에 대해 처리
        print(f"\n총 {len(products)}개의 제품 발견")

        for i, product in enumerate(products[:5]):  # 최대 5개만 처리
            try:
                print(f"\n[{i+1}/{min(len(products), 5)}] 제품 처리 중...")

                # 이미지 래퍼 찾기
                img_wrap = None
                possible_img_selectors = [
                    "CommonPcListUnitProduct_img_wrap__4haVO",
                    "[class*='img_wrap']",
                    "[class*='image']",
                    ".product-image"
                ]

                for selector in possible_img_selectors:
                    try:
                        if selector.startswith('['):
                            img_wrap = product.find_element(By.CSS_SELECTOR, selector)
                        else:
                            img_wrap = product.find_element(By.CLASS_NAME, selector)

                        if img_wrap:
                            break
                    except:
                        continue

                if not img_wrap:
                    # 제품 내부의 a 태그 직접 찾기
                    try:
                        link_element = product.find_element(By.TAG_NAME, "a")
                    except:
                        print("링크 요소를 찾을 수 없음")
                        continue
                else:
                    # 이미지 래퍼 내부의 a 태그 찾기
                    try:
                        link_element = img_wrap.find_element(By.TAG_NAME, "a")
                    except:
                        print("이미지 래퍼 내부 링크를 찾을 수 없음")
                        continue

                # 링크 정보 가져오기
                href = link_element.get_attribute("href")
                data_contents = link_element.get_attribute("data-contents")

                print(f"링크 URL: {href}")
                print(f"Data Contents: {data_contents}")

                if href:
                    # 스크롤하여 요소가 보이도록 함
                    driver.execute_script("arguments[0].scrollIntoView(true);", link_element)
                    time.sleep(1)

                    # 클릭 시도
                    try:
                        # JavaScript를 사용한 클릭
                        driver.execute_script("arguments[0].click();", link_element)
                        print("✓ 링크 클릭 성공 (JavaScript)")

                        # 새 페이지/탭 로딩 대기
                        time.sleep(3)

                        # 새 탭이 열렸는지 확인
                        if len(driver.window_handles) > 1:
                            # 새 탭으로 전환
                            driver.switch_to.window(driver.window_handles[-1])

                            # 페이지 정보 수집
                            page_title = driver.title
                            current_url = driver.current_url
                            print(f"새 페이지 제목: {page_title}")
                            print(f"새 페이지 URL: {current_url}")

                            # 제품 상세 정보 수집 시도
                            try:
                                product_details = extract_product_details(driver)
                                if product_details:
                                    print("제품 상세 정보:")
                                    for key, value in product_details.items():
                                        print(f"  {key}: {value}")
                            except Exception as e:
                                print(f"제품 상세 정보 수집 중 오류: {e}")

                            time.sleep(3)

                            # 탭 닫기
                            driver.close()

                            # 원래 탭으로 돌아가기
                            driver.switch_to.window(driver.window_handles[0])
                        else:
                            # 같은 탭에서 페이지 변경
                            page_title = driver.title
                            current_url = driver.current_url
                            print(f"페이지 제목: {page_title}")
                            print(f"현재 URL: {current_url}")

                            # 뒤로 가기
                            driver.back()
                            time.sleep(2)

                    except Exception as e:
                        print(f"클릭 실패: {e}")
                        # ActionChains를 사용한 클릭 시도
                        try:
                            actions = ActionChains(driver)
                            actions.move_to_element(link_element).click().perform()
                            print("✓ 링크 클릭 성공 (ActionChains)")
                        except Exception as e2:
                            print(f"ActionChains 클릭도 실패: {e2}")

                time.sleep(2)

            except Exception as e:
                print(f"제품 {i+1} 처리 중 오류: {e}")
                continue

    except Exception as e:
        print(f"전체 프로세스 중 오류 발생: {e}")

    finally:
        print("\n=== 브라우저 종료 ===")
        driver.quit()

def extract_product_details(driver):
    """제품 상세 페이지에서 정보 추출"""
    details = {}

    try:
        # 제품명
        try:
            product_name = driver.find_element(By.CSS_SELECTOR, "h1, .product-name, [class*='title'], [class*='name']").text
            details['제품명'] = product_name
        except:
            pass

        # 가격
        try:
            price = driver.find_element(By.CSS_SELECTOR, ".price, [class*='price'], [class*='cost']").text
            details['가격'] = price
        except:
            pass

        # 모델명
        try:
            model = driver.find_element(By.CSS_SELECTOR, ".model, [class*='model'], [class*='code']").text
            details['모델명'] = model
        except:
            pass

    except Exception as e:
        print(f"제품 정보 추출 중 오류: {e}")

    return details

def analyze_page_structure(url):
    """페이지 구조 분석 함수"""
    driver = setup_chrome_driver()

    try:
        print(f"페이지 구조 분석 중: {url}")
        driver.get(url)
        time.sleep(3)

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 모든 클래스명 찾기
        all_classes = set()
        for element in soup.find_all(True):
            classes = element.get('class', [])
            if classes:
                all_classes.update(classes)

        # 제품 관련 클래스 필터링
        product_classes = [cls for cls in all_classes if any(keyword in cls.lower() for keyword in ['product', 'item', 'list', 'unit'])]

        print("발견된 제품 관련 클래스:")
        for cls in sorted(product_classes):
            print(f"  {cls}")

        # 링크 요소 분석
        links = soup.find_all('a', href=True)
        print(f"\n총 {len(links)}개의 링크 발견")

        product_links = []
        for link in links:
            href = link.get('href', '')
            if any(keyword in href.lower() for keyword in ['product', 'dryer', 'item']):
                product_links.append(href)

        print(f"제품 관련 링크 {len(product_links)}개:")
        for link in product_links[:10]:  # 최대 10개만 출력
            print(f"  {link}")

    except Exception as e:
        print(f"페이지 구조 분석 중 오류: {e}")

    finally:
        driver.quit()

# 실행
if __name__ == "__main__":
    print("=== LG 전자 건조기 페이지 웹 스크래핑 시작 ===")

    # 페이지 구조 분석 (선택사항)
    # analyze_page_structure("https://www.lge.co.kr/category/dryers")

    # 메인 스크래핑 및 클릭 실행
    extract_and_click_lg_dryer_links()

    print("\n=== 작업 완료 ===")

=== LG 전자 건조기 페이지 웹 스크래핑 시작 ===
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:4 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,214 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,039 B in 2s (1,699 B/s)
Reading package lists... Done
W: Skipping acquire of configured fil

In [11]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import platform
import os

class LGDryerScraper:
    def __init__(self, headless=False):
        """
        LG 건조기 스크래퍼 초기화

        Args:
            headless (bool): 헤드리스 모드 실행 여부
        """
        self.driver = None
        self.wait = None
        self.headless = headless
        self.scraped_data = []

    def setup_driver(self):
        """Chrome 드라이버 설정"""
        chrome_options = Options()

        # 구글 코랩 환경 감지
        if 'COLAB_GPU' in os.environ:
            print("구글 코랩 환경 감지")
            # 코랩용 설정
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--remote-debugging-port=9222')
        else:
            print("로컬 환경 감지")
            if self.headless:
                chrome_options.add_argument('--headless')

        # 공통 설정
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        chrome_options.add_argument('--window-size=1920,1080')

        try:
            self.driver = webdriver.Chrome(options=chrome_options)
            self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            self.wait = WebDriverWait(self.driver, 10)
            print("Chrome 드라이버 설정 완료")
        except Exception as e:
            print(f"드라이버 설정 실패: {e}")
            raise

    def open_lg_dryer_page(self):
        """LG 건조기 카테고리 페이지 열기"""
        try:
            print("LG 건조기 페이지 열기...")
            self.driver.get("https://www.lge.co.kr/category/dryers")
            time.sleep(3)
            print("페이지 로딩 완료")
        except Exception as e:
            print(f"페이지 열기 실패: {e}")
            raise

    def find_and_click_dryer_products(self):
        """건조기 제품 찾기 및 클릭"""
        try:
            print("건조기 제품 찾는 중...")

            # 제품 리스트 컨테이너 찾기
            product_list = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul[role='list']"))
            )

            # 모든 제품 아이템 찾기
            product_items = product_list.find_elements(By.CSS_SELECTOR, "li.CommonPcListUnitProduct_unit__h6bSJ")
            print(f"총 {len(product_items)}개의 제품 발견")

            # 각 제품에 대해 처리
            for i, product_item in enumerate(product_items):
                try:
                    # 제품 이미지 래퍼 찾기
                    img_wrap = product_item.find_element(By.CSS_SELECTOR, "div.CommonPcListUnitProduct_img_wrap__4haVO")

                    # 트롬 오브제컬렉션 건조기 링크 찾기
                    product_link = img_wrap.find_element(By.CSS_SELECTOR, "a[data-contents*='트롬 오브제컬렉션 건조기'], a[data-contents*='건조기']")

                    product_name = product_link.get_attribute("data-contents")
                    print(f"제품 {i+1}: {product_name}")

                    # 새 탭에서 열기 위해 Ctrl+Click (macOS에서는 Cmd+Click)
                    if platform.system() == "Darwin":  # macOS
                        self.driver.execute_script("arguments[0].click();", product_link)
                    else:
                        product_link.click()

                    # 새 탭으로 전환
                    self.driver.switch_to.window(self.driver.window_handles[-1])
                    time.sleep(2)

                    # 제품 상세 정보 스크래핑
                    self.scrape_product_details()

                    # 원래 탭으로 돌아가기
                    self.driver.close()
                    self.driver.switch_to.window(self.driver.window_handles[0])
                    time.sleep(1)

                except Exception as e:
                    print(f"제품 {i+1} 처리 중 오류: {e}")
                    continue

        except Exception as e:
            print(f"제품 찾기/클릭 실패: {e}")
            raise

    def scrape_product_details(self):
        """제품 상세 정보 스크래핑"""
        product_data = {}

        try:
            print("제품 상세 정보 스크래핑 중...")

            # 3. 제품명 스크래핑
            try:
                product_name_element = self.wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.product-detail-info div.product-top-section div.product-name h2.name"))
                )
                product_data['product_name'] = product_name_element.text.strip()
                print(f"제품명: {product_data['product_name']}")
            except Exception as e:
                print(f"제품명 스크래핑 실패: {e}")
                product_data['product_name'] = "N/A"

            # 4. 버튼 텍스트 스크래핑
            try:
                button_elements = self.driver.find_elements(By.CSS_SELECTOR, "div.product-detail-info div.product-top-section button[type='button'] span.blind")
                button_texts = [btn.text.strip() for btn in button_elements if btn.text.strip()]
                product_data['button_texts'] = button_texts
                print(f"버튼 텍스트: {button_texts}")
            except Exception as e:
                print(f"버튼 텍스트 스크래핑 실패: {e}")
                product_data['button_texts'] = []

            # 5. 가격 정보 스크래핑
            try:
                price_elements = self.driver.find_elements(By.CSS_SELECTOR, "div.product-detail-option div.info-accordion-wrap ul li.lists.member div.price-detail ul li.price-detail-item#benefitPrice dl.price-info dd.content a span.price")
                prices = [price.text.strip() for price in price_elements if price.text.strip()]
                product_data['prices'] = prices
                print(f"가격 정보: {prices}")
            except Exception as e:
                print(f"가격 정보 스크래핑 실패: {e}")
                product_data['prices'] = []

            # 6. 제품스펙 상세정보 펼치기 클릭
            try:
                expand_button = self.driver.find_element(By.CSS_SELECTOR, "div.btn_collapse_more a[title*='제품스펙 상세정보 펼치기']")
                self.driver.execute_script("arguments[0].click();", expand_button)
                time.sleep(2)
                print("제품스펙 상세정보 펼치기 완료")
            except Exception as e:
                print(f"제품스펙 펼치기 실패: {e}")

            # 7. 제품 스펙 상세 정보 스크래핑
            try:
                spec_items = self.driver.find_elements(By.CSS_SELECTOR, "div.prod-spec-detail div.box div.cls ul li dl")
                specs = {}

                for spec_item in spec_items:
                    try:
                        dt_element = spec_item.find_element(By.TAG_NAME, "dt")
                        dd_element = spec_item.find_element(By.TAG_NAME, "dd")

                        spec_key = dt_element.text.strip()
                        spec_value = dd_element.text.strip()

                        if spec_key and spec_value:
                            specs[spec_key] = spec_value
                    except Exception as e:
                        continue

                product_data['specifications'] = specs
                print(f"제품 스펙: {len(specs)}개 항목")

            except Exception as e:
                print(f"제품 스펙 스크래핑 실패: {e}")
                product_data['specifications'] = {}

            # 스크래핑된 데이터 저장
            self.scraped_data.append(product_data)

        except Exception as e:
            print(f"제품 상세 정보 스크래핑 실패: {e}")

    def save_data_to_json(self, filename="lg_dryer_data.json"):
        """스크래핑된 데이터를 JSON 파일로 저장"""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.scraped_data, f, ensure_ascii=False, indent=2)
            print(f"데이터가 {filename}에 저장되었습니다.")
        except Exception as e:
            print(f"데이터 저장 실패: {e}")

    def print_scraped_data(self):
        """스크래핑된 데이터 출력"""
        print("\n=== 스크래핑된 데이터 ===")
        for i, data in enumerate(self.scraped_data, 1):
            print(f"\n--- 제품 {i} ---")
            print(f"제품명: {data.get('product_name', 'N/A')}")
            print(f"버튼 텍스트: {data.get('button_texts', [])}")
            print(f"가격 정보: {data.get('prices', [])}")
            print(f"제품 스펙 항목 수: {len(data.get('specifications', {}))}")

            if data.get('specifications'):
                print("주요 스펙:")
                for key, value in list(data['specifications'].items())[:5]:  # 상위 5개만 출력
                    print(f"  {key}: {value}")

    def run(self):
        """전체 스크래핑 프로세스 실행"""
        try:
            print("=== LG 건조기 스크래핑 시작 ===")

            # 드라이버 설정
            self.setup_driver()

            # 페이지 열기
            self.open_lg_dryer_page()

            # 제품 찾기 및 클릭
            self.find_and_click_dryer_products()

            # 데이터 출력
            self.print_scraped_data()

            # 데이터 저장
            self.save_data_to_json()

            print("\n=== 스크래핑 완료 ===")

        except Exception as e:
            print(f"스크래핑 프로세스 오류: {e}")
        finally:
            if self.driver:
                self.driver.quit()
                print("브라우저 종료")

def main():
    """메인 함수"""
    # 구글 코랩 환경에서 필요한 설정
    if 'COLAB_GPU' in os.environ:
        print("구글 코랩 환경 설정 중...")
        os.system("apt-get update")
        os.system("apt install chromium-chromedriver")
        os.system("cp /usr/lib/chromium-browser/chromedriver /usr/bin")

        # 추가 패키지 설치
        os.system("pip install selenium")

    # 스크래퍼 실행
    scraper = LGDryerScraper(headless=False)  # GUI 모드로 실행 (코랩에서는 자동으로 헤드리스)
    scraper.run()

if __name__ == "__main__":
    main()

구글 코랩 환경 설정 중...
=== LG 건조기 스크래핑 시작 ===
구글 코랩 환경 감지
Chrome 드라이버 설정 완료
LG 건조기 페이지 열기...
페이지 로딩 완료
건조기 제품 찾는 중...
총 30개의 제품 발견
제품 1: LG 트롬 오브제컬렉션 건조기
제품 상세 정보 스크래핑 중...
제품명: LG 트롬 오브제컬렉션 건조기 21kg 1등급
버튼 텍스트: ['모델명 복사']
가격 정보: ['1,878,600원']
제품스펙 상세정보 펼치기 완료
제품 스펙: 0개 항목
제품 1 처리 중 오류: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
#0 0x5c309ca6f23a <unknown>
#1 0x5c309c519903 <unknown>
#2 0x5c309c55ccaf <unknown>
#3 0x5c309c5910a6 <unknown>
#4 0x5c309c58bba7 <unknown>
#5 0x5c309c58ace5 <unknown>
#6 0x5c309c4e1928 <unknown>
#7 0x5c309ca33b4b <unknown>
#8 0x5c309ca37929 <unknown>
#9 0x5c309ca1a929 <unknown>
#10 0x5c309ca384e8 <unknown>
#11 0x5c309c9ff0df <unknown>
#12 0x5c309c4df742 <unknown>
#13 0x7a4324db2d90 <unknown>

제품 2 처리 중 오류: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdr

In [14]:
# 구글 코랩에서 크롬 브라우저로 웹페이지 열기 자동화

# 1. 필요한 패키지 설치
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!pip install selenium

# 2. 필요한 라이브러리 import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 3. 크롬 브라우저 설정
def setup_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 헤드리스 모드 (브라우저 창 없이 실행)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    driver = webdriver.Chrome(options=chrome_options)
    return driver

# 4. LG 전자 웹페이지 URL 리스트
urls = [
    "https://www.lge.co.kr/category/wash-tower",
    "https://www.lge.co.kr/category/wash-combo",
    "https://www.lge.co.kr/category/washing-machines",
    "https://www.lge.co.kr/category/dryers"
]

# 5. 웹페이지 열기 함수
def open_webpages(urls):
    driver = setup_chrome_driver()

    try:
        for i, url in enumerate(urls):
            print(f"\n{i+1}. 웹페이지 열기: {url}")

            # 웹페이지 열기
            driver.get(url)

            # 페이지 로딩 대기
            time.sleep(3)

            # 페이지 타이틀 출력
            print(f"   페이지 타이틀: {driver.title}")

            # 페이지 스크린샷 저장 (선택사항)
            screenshot_name = f"screenshot_{i+1}.png"
            driver.save_screenshot(screenshot_name)
            print(f"   스크린샷 저장: {screenshot_name}")

            # 각 페이지 사이 대기 시간
            time.sleep(2)

    except Exception as e:
        print(f"오류 발생: {e}")

    finally:
        # 브라우저 종료
        driver.quit()
        print("\n브라우저 종료 완료")

# 6. 메인 실행
if __name__ == "__main__":
    print("LG 전자 세탁기/건조기 페이지 자동화 시작")
    print("=" * 50)

    open_webpages(urls)

    print("\n모든 웹페이지 열기 완료!")

# 7. 개별 페이지 테스트 함수 (선택사항)
def test_single_page(url):
    """단일 페이지 테스트용 함수"""
    driver = setup_chrome_driver()

    try:
        print(f"테스트 페이지: {url}")
        driver.get(url)
        time.sleep(3)

        print(f"페이지 타이틀: {driver.title}")
        print(f"현재 URL: {driver.current_url}")

        # 페이지 소스 일부 확인
        print(f"페이지 소스 길이: {len(driver.page_source)} 문자")

        driver.save_screenshot("test_screenshot.png")
        print("테스트 스크린샷 저장 완료")

    except Exception as e:
        print(f"테스트 오류: {e}")

    finally:
        driver.quit()

# 개별 테스트 실행 예시
# test_single_page("https://www.lge.co.kr/category/wash-tower")

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading

In [15]:
# 라이브러리 설치 (최초 실행 시만)
!pip install selenium webdriver-manager beautifulsoup4

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# 크롬 드라이버 설정
def get_driver():
    options = Options()
    options.add_argument('--headless')  # 백그라운드 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL 리스트
urls = [
    "https://www.lge.co.kr/category/wash-tower",
    "https://www.lge.co.kr/category/wash-combo",
    "https://www.lge.co.kr/category/washing-machines",
    "https://www.lge.co.kr/category/dryers"
]

# 결과 저장용
result_dict = {}

# 스크래핑 실행
driver = get_driver()

for url in urls:
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    container = soup.select_one('.CommonPcListUnitProduct_list_unit_product__AeuOT')

    if container:
        li_tags = container.find_all('li', attrs={'data-ec-product': True})
        data_list = [li['data-ec-product'] for li in li_tags]
        result_dict[url] = data_list
        print(f"\n✅ {url} - {len(data_list)}개 항목 추출 완료")
    else:
        print(f"\n❌ {url} - 대상 클래스가 존재하지 않음")

driver.quit()

# 결과 예시 출력
for url, data in result_dict.items():
    print(f"\n📌 URL: {url}")
    for i, item in enumerate(data[:3]):  # 각 URL당 최대 3개 샘플 출력
        print(f"{i+1}. {item}")



✅ https://www.lge.co.kr/category/wash-tower - 0개 항목 추출 완료

✅ https://www.lge.co.kr/category/wash-combo - 26개 항목 추출 완료

✅ https://www.lge.co.kr/category/washing-machines - 30개 항목 추출 완료

✅ https://www.lge.co.kr/category/dryers - 30개 항목 추출 완료

📌 URL: https://www.lge.co.kr/category/wash-tower

📌 URL: https://www.lge.co.kr/category/wash-combo
1. {"model_name":"LG 트롬 오브제컬렉션 워시콤보","model_id":"MD10335844","model_sku":"FH25KA.AKOR","model_gubun":"일반제품","base_price":"3,700,000","price":"3,273,600","discounted_price":"3,700,000","grand_price":"4,440,000","brand":"LG","category":"생활가전/워시콤보/워시콤보","ct_id":"CT50250001","model_super_category":"생활가전","model_category":"워시콤보","model_sub_category":"워시콤보","delivery_badge":"N","price_badge":"Y"}
2. {"model_name":"LG 트롬 오브제컬렉션 워시콤보","model_id":"MD10480828","model_sku":"FH25ESE.AKOR","model_gubun":"일반제품","base_price":"4,440,000","price":"3,961,800","discounted_price":"4,440,000","grand_price":"4,540,000","brand":"LG","category":"생활가전/워시콤보/워시콤보","ct_id":"CT50

In [ ]:
# 셀 1: 필수 패키지 설치
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


In [25]:
def scroll_through_all_lis(driver):
    """모든 li 요소를 순차적으로 스크롤하여 lazy loading을 유도"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 모든 li를 대상으로 하나씩 스크롤
        lis = driver.find_elements("css selector", "section.CommonPcListUnitProduct_list_unit_product__AeuOT ul[role='list'] > li")
        for li in lis:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", li)
                time.sleep(0.3)  # 로딩 대기
            except:
                continue

        # 새 높이를 확인해 더 로딩되는지 판단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


In [26]:
# 셀 2: Selenium 설정 및 데이터 수집
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# ✅ 크롤링 대상 URL 리스트
urls = [
    "https://www.lge.co.kr/category/wash-tower",
    "https://www.lge.co.kr/category/wash-combo",
    "https://www.lge.co.kr/category/washing-machines",
    "https://www.lge.co.kr/category/dryers"
]

# ✅ Chrome 옵션 설정 (Colab 환경)
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ✅ 크롬 드라이버 실행
driver = webdriver.Chrome(options=options)

# ✅ 스크래핑 함수 정의
def scrape_data_ec_product(url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    scroll_through_all_lis(driver)  # ✅ 실제 li를 보이게 함

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    section = soup.find('section', class_="CommonPcListUnitProduct_list_unit_product__AeuOT")
    data_list = []

    if section:
        ul = section.find('ul', role='list')
        if ul:
            lis = ul.find_all('li')
            for li in lis:
                data = li.get('data-ec-product')
                if data:
                    data_list.append(data)
    return data_list

# ✅ 전체 URL 대상 스크래핑 수행
all_data = {}
for url in urls:
    print(f"크롤링 중: {url}")
    data = scrape_data_ec_product(url)
    all_data[url] = data
    print(f"수집된 항목 수: {len(data)}")

driver.quit()


크롤링 중: https://www.lge.co.kr/category/wash-tower
수집된 항목 수: 89
크롤링 중: https://www.lge.co.kr/category/wash-combo
수집된 항목 수: 26
크롤링 중: https://www.lge.co.kr/category/washing-machines
수집된 항목 수: 266
크롤링 중: https://www.lge.co.kr/category/dryers
수집된 항목 수: 102


In [27]:
# 셀 3: 결과 확인
import json

# ✅ 결과를 JSON 형식으로 출력
for url, items in all_data.items():
    print(f"\n📌 URL: {url}")
    for item in items:
        print(json.loads(item))  # JSON 문자열을 dict로 변환하여 출력


📌 URL: https://www.lge.co.kr/category/wash-tower
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09942859', 'model_sku': 'WL21WDU.AKOR', 'model_gubun': '일반제품', 'base_price': '3,100,000', 'price': '2,715,600', 'discounted_price': '3,100,000', 'grand_price': '3,990,000', 'brand': 'LG', 'category': '생활가전/워시타워/워시타워', 'ct_id': 'CT50000110', 'model_super_category': '생활가전', 'model_category': '워시타워', 'model_sub_category': '워시타워', 'delivery_badge': 'N', 'price_badge': 'Y'}
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09942827', 'model_sku': 'WL21EGZU.AKOR', 'model_gubun': '일반제품', 'base_price': '3,260,000', 'price': '2,864,400', 'discounted_price': '3,260,000', 'grand_price': '4,290,000', 'brand': 'LG', 'category': '생활가전/워시타워/워시타워', 'ct_id': 'CT50000110', 'model_super_category': '생활가전', 'model_category': '워시타워', 'model_sub_category': '워시타워', 'delivery_badge': 'N', 'price_badge': 'Y'}
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09900826', 'model_sku': 'W20WHN.AKOR', 'model_gubun'

In [ ]:
import requests
from bs4 import BeautifulSoup

def fetch_image_sources(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        section = soup.select_one('section.CommonPcListUnitProduct_list_unit_product__AeuOT ul[role="list"]')
        if not section:
            return []

        img_tags = section.select('div.slide-content.pdp-visual.ui_carousel_list.ui_static.draggable ul.slide-track.pdp-visual-list.ui_carousel_track.ui_static li a img')
        return [img['src'] for img in img_tags if img.get('src')]

    except (requests.RequestException, AttributeError):
        return []

for url in urls:
    for src in fetch_image_sources(url):
        print(src)
